**WELCOME TO PART II HACKERS**

> We will quickly introduce you to the various packages you will need to help you develop the Geographical Information System application/website/dashboard using Colab


---



1.   GEEMAP is a Python package for interactive geospatial analysis and visualization with Google Earth Engine (GEE), which is a cloud computing platform with a multi-petabyte catalog of satellite imagery and geospatial datasets. You will need to install it to interact with readily available satellite image data.




In [ ]:
!pip install geemap

In [ ]:
!pip install PyCRS

In [ ]:
import geemap
import ee
import os
import geopandas
from geemap import geojson_to_ee, ee_to_geojson


2.   Authentication with Earth Engine is required to connect your project with Earth Engine



Run the ee.Authenticate function to authenticate your access to Earth Engine servers and ee.Initialize to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. More at: https://developers.google.com/earth-engine/guides/python_install

In case ee.Initialize() is not running, you need to create your own cloud EE project and enable API direclty to allow Colab to use EE features through this link: https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup (follow the link on the section "Create a Cloud project" then proceed with the next section "Enable the Earth Engine API")

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize (project='atd-gc')

Alternatively: Open GEE Code Editor https://code.earthengine.google.com/, set up your own project manually (set up a non commerciable project) and enable permission for the GEE APIs for Colab.


```
ee.Initialize (project='your project name')
```





---



Now let's start with basic vizualization process, first you will be asked to upload the batch of shapefiles provided to you in your local file folder in collab. The Brazil.shp file serves as the official country boundary of our area of interest.

In [ ]:
brazil_shapefile = geemap.shp_to_ee('content/Brazil.shp')

Next, let's see how the land cover was in January 1st 2004 in Brazil using the MODIS MCD12Q1 Type 1 Land Cover

In [ ]:
Map = geemap.Map()

landcover = ee.Image('MODIS/006/MCD12Q1/2004_01_01').select('LC_Type1')

igbpLandCoverVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a',
        '086a10',
        '54a708',
        '78d203',
        '009900',
        'c6b044',
        'dcd159',
        'dade48',
        'fbff13',
        'b6ff05',
        '27ff87',
        'c24f44',
        'a5a5a5',
        'ff6d4c',
        '69fff8',
        'f9ffa4',
        '1c0dff',
    ],
}
brazil_lc = landcover.clip(brazil_shapefile)
Map.setCenter(-55, -10, 4)
Map.addLayer(brazil_lc, igbpLandCoverVis, 'MODIS Land Cover')

Map

In [ ]:
Map.add_legend(builtin_legend='MODIS/051/MCD12Q1')

Next, we can compute basic statistics of how much hectares of each LC type was in Brazil in 2004. This line will allow you to donwload the results in your colab directory

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
global_stats = os.path.join(out_dir, 'global_stats_sum.csv')

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(
    landcover,
    brazil_shapefile,
    global_stats,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=2,
)

Better: you can just count the number of pixel for each class. As a pixel for a MODIS image is 250 m x 250 m, the area for one class will be number of pixel x 250 x 250

In [ ]:
# Calculate pixel areas
areaImage = ee.Image.pixelArea().addBands(lc14)

# Reduce the region to get summed areas
areas = areaImage.reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='class'),
    geometry=fc.geometry(),
    scale=500,
    maxPixels=1e10
)

# Print the result
print(areas.getInfo())

Now, let's pull a time series vizualization of the landcover in Brazil



1.   This is the long code to help you understand the process



In [ ]:
Map = geemap.Map()

lc03 = ee.Image('MODIS/006/MCD12Q1/2003_01_01').select('LC_Type1')
lc14 = ee.Image('MODIS/006/MCD12Q1/2014_01_01').select('LC_Type1')
lc15 = ee.Image('MODIS/006/MCD12Q1/2015_01_01').select('LC_Type1')
lc16 = ee.Image('MODIS/006/MCD12Q1/2016_01_01').select('LC_Type1')
lc17 = ee.Image('MODIS/006/MCD12Q1/2017_01_01').select('LC_Type1')
lc18 = ee.Image('MODIS/006/MCD12Q1/2018_01_01').select('LC_Type1')
lc19 = ee.Image('MODIS/006/MCD12Q1/2019_01_01').select('LC_Type1')
lc20 = ee.Image('MODIS/006/MCD12Q1/2020_01_01').select('LC_Type1')

igbpLandCoverVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a',
        '086a10',
        '54a708',
        '78d203',
        '009900',
        'c6b044',
        'dcd159',
        'dade48',
        'fbff13',
        'b6ff05',
        '27ff87',
        'c24f44',
        'a5a5a5',
        'ff6d4c',
        '69fff8',
        'f9ffa4',
        '1c0dff',
    ],
}
brazil_lc03 = lc03.clip(brazil_shapefile)
brazil_lc14 = lc14.clip(brazil_shapefile)
brazil_lc15 = lc15.clip(brazil_shapefile)
brazil_lc16 = lc16.clip(brazil_shapefile)
brazil_lc17 = lc17.clip(brazil_shapefile)
brazil_lc18 = lc18.clip(brazil_shapefile)
brazil_lc19 = lc19.clip(brazil_shapefile)
brazil_lc20 = lc20.clip(brazil_shapefile)

Map.setCenter(-55, -10, 4)
Map.addLayer(brazil_lc14, igbpLandCoverVis, 'MODIS Land Cover 2014')
Map.addLayer(brazil_lc15, igbpLandCoverVis, 'MODIS Land Cover 2015')
Map.addLayer(brazil_lc16, igbpLandCoverVis, 'MODIS Land Cover 2016')
Map.addLayer(brazil_lc17, igbpLandCoverVis, 'MODIS Land Cover 2017')
Map.addLayer(brazil_lc18, igbpLandCoverVis, 'MODIS Land Cover 2018')
Map.addLayer(brazil_lc19, igbpLandCoverVis, 'MODIS Land Cover 2019')
Map.addLayer(brazil_lc20, igbpLandCoverVis, 'MODIS Land Cover 2020')

Map.addLayerControl()

Map

NEXT is how you import and add population density to the previous map

In [ ]:
dataset = ee.ImageCollection('WorldPop/GP/100m/pop')

# Visualization parameters
visualization = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}

# Set center of the map
center = [34.769, 113.643]
zoom = 7

# Add the layer to the map
image = dataset.mean()
Map.addLayer(image, visualization, 'Population')




2.   This is how it looks like when it is more intricate (you might laugh cause it is not at all intricate for you)


In [ ]:
# Load MODIS land cover images for multiple years
years = range(2002, 2021)  # 2002 to 2020
lc_images = []

for year in years:
    lc_image = ee.Image(f"MODIS/006/MCD12Q1/{year}_01_01").select('LC_Type1')
    lc_images.append(lc_image)

# Visualization parameters
igbpLandCoverVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044',
        'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
        'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff'
    ]
}

# Clip the land cover images to Brazil
brazil_lc_images = [lc.clip(brazil_shapefile) for lc in lc_images]

# Display the land cover images for each year
Map = geemap.Map(center=[-10, -55], zoom=4)

for i, lc_image in enumerate(brazil_lc_images):
    Map.addLayer(lc_image, igbpLandCoverVis, f'MODIS Land Cover {years[i]}')

legend_keys = ['Evergreen Needleleaf Forests', 'Evergreen Broadleaf Forests', 'Deciduous Needleleaf Forests', 'Deciduous Broadleaf Forests',
               'Mixed Forests', 'Closed Shrublands', 'Open Shrublands', 'Woody Savannas',
               'Savannas', 'Grasslands', 'Permanent Wetlands', 'Croplands', 'Urban and Built-up',
               'Cropland/Natural Vegetation Mosaics', 'Permanent Snow and Ice', 'Barren', 'Water Bodies']
igbpLandCoverVisu = ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044',
        'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
        'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']

Map.add_legend(keys=legend_keys, colors=igbpLandCoverVisu, position='bottomright')
Map.addLayerControl()

Map


You can also suggest a side by side vizualization of the land cover, allowing one to vizaully assess the extent of degradation over time.

In [ ]:
Map = geemap.Map(center=[-9, -51], zoom=10)

left_layer = geemap.ee_tile_layer(brazil_lc03, igbpLandCoverVis, "MODIS")
right_layer = geemap.ee_tile_layer(brazil_lc20, igbpLandCoverVis, "MODIS")
Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="Land cover 2003", builtin_legend='MODIS/006/MCD12Q1', position='bottomleft'
)
Map.add_legend(
    title="Land cover 2020",
    builtin_legend='MODIS/006/MCD12Q1',
    position='bottomright',
)


Map

This part is more fun!!! Using the timeseries tool to compare 20 years of landcover against each other.

In [ ]:
col = ee.ImageCollection('MODIS/006/MCD12Q1').select('LC_Type1');

In [ ]:
# A FeatureCollection defining Brazil boundary.
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    'country_na == "Brazil"'
)

# Clip the DEM by the Southeast Asia boundary FeatureCollection.
# Iterate over the ImageCollection and clip each image to the FeatureCollection.
col_clip = col.map(lambda img: img.clipToCollection(fc))
DOY = col_clip.filterDate('2002-01-01', '2021-01-01');

In [ ]:
distinctDOY = col.filterDate('2002-01-01', '2021-01-01');

Check the number of image you have between that range:

In [ ]:
num_images = distinctDOY.size().getInfo()

print("Number of images in the collection:", num_images)

In [ ]:
layer_names = ['MODIS ' + str(year) for year in range(2002, 2021)]
print(layer_names)

In [ ]:
Map = geemap.Map(center=[-9, -51], zoom=10)
Map.add_basemap('HYBRID')
# Map.centerObject(brazil_shapefile)

images = geemap.modis_timeseries(asset_id='MODIS/006/MCD12Q1', band_name='LC_Type1', roi=brazil_shapefile,
                                 start_year = 2003, end_year = 2021,
                                 start_date= '01-01', end_date= '12-31')
Map.ts_inspector(left_ts=DOY, left_names=layer_names, left_vis=igbpLandCoverVis, left_index=0,
                 right_ts=DOY, right_names=layer_names, right_vis=igbpLandCoverVis, right_index=-1,
                 width='130px', date_format='YYYY', add_close_button=False)
Map.add_legend(title="MODIS Land Cover", builtin_legend='MODIS/006/MCD12Q1')
Map.addLayerControl()

Map

One way to calculate area of a specific Image in Brazil

THIS IS THE CODE TO VIZUALIZE THE BURNED AREAS

In [ ]:
# Define the dataset and filter by date
dataset = ee.ImageCollection('MODIS/061/MCD64A1').filter(ee.Filter.date('2017-01-01', '2018-05-01'))
burnedArea = dataset.select('BurnDate')

# A FeatureCollection defining Brazil boundary.
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    'country_na == "Brazil"'
)

# Clip the burned Area by the Brazil boundary FeatureCollection.
# Iterate over the ImageCollection and clip each image to the FeatureCollection.
ba_clip = burnedArea.map(lambda img: img.clipToCollection(fc))


# Visualization parameters
burnedAreaVis = {
    'min': 30.0,
    'max': 341.0,
    'palette': ['4e0400', '951003', 'c61503', 'ff1901']
}

# Create a map
Map = geemap.Map(center=[-10, -55], zoom=4)

# Add burned area layer to the map
Map.addLayer(ba_clip, burnedAreaVis, 'Burned Area')
Map.addLayer(brazil_shapefile, name='Brazil',opacity=0.5)
Map.addLayerControl()

# Display the map
Map

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install altair

In [ ]:
!pip install streamlit-elements==0.1.*

In [ ]:
!pip install geopandas

In [ ]:
!pip install panel

In [ ]:
!npm install localtunnel

We used Streamlit to create our dashboard. Streamlit is an open-source Python library, which helps in creating web apps with interactive and beautiful visuals such as charts, maps, and tables. It is also compatible with Google Earth Engine.

In [ ]:
%%writefile app.py

import streamlit as st
import geemap
import ee

import streamlit as st
import pandas as pd
import numpy as np
import altair
import geopandas as gpd

### You should authenticate to your Google account and initialize your own project in Google Earth Engine
ee.Authenticate()
ee.Initialize(project='atd-gc')

st.title('Your Dashboard')

# We created 4 columns for the main metrics
# These columns can be added/removed depending on the need for more/less metrics
col1, col2, col3, col4 = st.columns(4)
col1.metric("Lowest Soil Moisture Rate", "25%", "+2%")
col2.metric("Number of High Risk Areas", "1,156", "+15.03%")
col3.metric("Number of Sensor Triggers", "367K", "-0.03%")
col4.metric("Unresolved Alerts", "23", "+2.02%")

col5, col6, col7, col8 = st.columns(4)
col5.metric("Total area protected", "30.62%", "+0.04%")
col6.metric("Evergreen forest area", "358mil", "-1.92%")
col7.metric("Number of fires YTD", "147K", "+0.13%")
col8.metric("Hectars burned down YTD", "10.7mil", "+2.02%")

# Placeholder for notifications and map (to be implemented)
st.subheader('Notifications')
st.text('Area 15B has been categorized...')
st.subheader('Visualization')


brazil_shapefile = geemap.shp_to_ee('content/Brazil.shp')


igbpLandCoverVis1 = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a',
        '086a10',
        '54a708',
        '78d203',
        '009900',
        'c6b044',
        'dcd159',
        'dade48',
        'fbff13',
        'b6ff05',
        '27ff87',
        'c24f44',
        'a5a5a5',
        'ff6d4c',
        '69fff8',
        'f9ffa4',
        '1c0dff',
    ],
}



gdf = gpd.read_file("content/wdpa_wdoecm_jan2024_public_bra_shp-polygons.shp")

# Convert GeoDataFrame to GEE FeatureCollection

gdf['geometry'] = gdf['geometry'].simplify(tolerance=500)  # Adjust 'tolerance' as needed

# Now that the geometries have been simplified, convert the entire GeoDataFrame to a GEE FeatureCollection
feature_collection = geemap.geopandas_to_ee(gdf)

# A FeatureCollection defining Brazil boundary
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))

dataset = ee.ImageCollection('MODIS/061/MCD64A1').filter(ee.Filter.date('2020-01-01', '2024-01-01'))
burnedArea = dataset.select('BurnDate')

# A FeatureCollection defining Brazil boundary.
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    'country_na == "Brazil"'
)

# Clip the burned Area by the Brazil boundary FeatureCollection.
# Iterate over the ImageCollection and clip each image to the FeatureCollection.
ba_clip = burnedArea.map(lambda img: img.clipToCollection(fc))


# Visualization parameters
burnedAreaVis2 = {
    'min': 30.0,
    'max': 341.0,
    'palette': ['4e0400', '951003', 'c61503', 'ff1901']
}


# Visualization parameters
ProtectedAreaVis2 = {
    'color': '#006400',  # This sets the outline color to green. Use hexadecimal colors.
    'opacity': 1,  # This sets the opacity for the outline.
}


paramVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a',
        '086a10',
        '54a708',
        '78d203',
        '009900',
        'c6b044',
        'dcd159',
        'dade48',
        'fbff13',
        'b6ff05',
        '27ff87',
        'c24f44',
        'a5a5a5',
        'ff6d4c',
        '69fff8',
        'f9ffa4',
        '1c0dff',
    ],
}



# We created 4 main tabs in our dashboard to show the effect of wildfires in Brazil
# The first tab shows the land cover across Brazil between years 2016-2020
# The second tab shows the burned areas
# The third tab shows both the population and the burned areas for every 5 years between 2000-2020.
# Possible correlations can be observed more clearly here.
# The fourth tab shows the change in land cover between the earlist and latest images acquired.

# Initialize tabs
tab1, tab2, tab3, tab4 = st.tabs(["Land Cover over Years", "Burned Areas and Protected Areas", "Population", "Land Cover Difference"])

# First map setup
with tab1:
    # Add the header to the tab
    st.header("Land Cover Changes Over Year")

    # Create a map
    Map1 = geemap.Map(center=[-10, -55], zoom=4)

    # Add Land Cover images to the map
    lc1 = ee.Image('MODIS/006/MCD12Q1/'+'2016_01_01').select('LC_Type1')
    lc2 = ee.Image('MODIS/006/MCD12Q1/'+'2017_01_01').select('LC_Type1')
    lc3 = ee.Image('MODIS/006/MCD12Q1/'+'2018_01_01').select('LC_Type1')
    lc4 = ee.Image('MODIS/006/MCD12Q1/'+'2019_01_01').select('LC_Type1')
    lc5 = ee.Image('MODIS/006/MCD12Q1/'+'2020_01_01').select('LC_Type1')

    brazil_lc1 = lc1.clip(brazil_shapefile)
    brazil_lc2 = lc2.clip(brazil_shapefile)
    brazil_lc3 = lc3.clip(brazil_shapefile)
    brazil_lc4 = lc4.clip(brazil_shapefile)
    brazil_lc5 = lc5.clip(brazil_shapefile)

    Map1.addLayer(brazil_lc1, igbpLandCoverVis1, 'MODIS Land Cover 2016')
    Map1.addLayer(brazil_lc2, igbpLandCoverVis1, 'MODIS Land Cover 2017')
    Map1.addLayer(brazil_lc3, igbpLandCoverVis1, 'MODIS Land Cover 2018')
    Map1.addLayer(brazil_lc4, igbpLandCoverVis1, 'MODIS Land Cover 2019')
    Map1.addLayer(brazil_lc5, igbpLandCoverVis1, 'MODIS Land Cover 2020')

    Map1.add_legend(
        title="Land cover",
        builtin_legend='MODIS/006/MCD12Q1'
    )

    # Show the map
    Map1.to_streamlit(height=700)

# Second map setup
with tab2:
    # Add the header to the tab
    st.header("Burned Areas and Protected Areas")

    # Create a map
    Map2 = geemap.Map(center=[-10, -55], zoom=4)

    # Add burned area layer to the map
    Map2.addLayer(feature_collection , ProtectedAreaVis2, 'Protected Areas')
    Map2.addLayer(ba_clip, burnedAreaVis2, 'Burned Area')

    # Show Map2
    Map2.to_streamlit(height=700)

# Third map setup
with tab3:
    st.header("Population 2000-2020")

    # A FeatureCollection defining Brazil boundary.
    fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
        'country_na == "Brazil"'
    )

    # Clip the burned Area by the Brazil boundary FeatureCollection.
    # Iterate over the ImageCollection and clip each image to the FeatureCollection.
    ba_clip = burnedArea.map(lambda img: img.clipToCollection(fc))

    ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Density')
    image1 = ee.Image('CIESIN/GPWv411/GPW_Population_Density/gpw_v4_population_density_rev11_2000_30_sec')
    image2 = ee.Image('CIESIN/GPWv411/GPW_Population_Density/gpw_v4_population_density_rev11_2005_30_sec')
    image3 = ee.Image('CIESIN/GPWv411/GPW_Population_Density/gpw_v4_population_density_rev11_2010_30_sec')
    image4 = ee.Image('CIESIN/GPWv411/GPW_Population_Density/gpw_v4_population_density_rev11_2015_30_sec')
    image5 = ee.Image('CIESIN/GPWv411/GPW_Population_Density/gpw_v4_population_density_rev11_2020_30_sec')

    brazil_pop1 = image1.clip(brazil_shapefile)
    brazil_pop2 = image2.clip(brazil_shapefile)
    brazil_pop3 = image3.clip(brazil_shapefile)
    brazil_pop4 = image4.clip(brazil_shapefile)
    brazil_pop5 = image5.clip(brazil_shapefile)

    # Initialize the map
    Map3 = geemap.Map(center=[-9, -51], zoom=4)

    # Add the Burned Area images to the map
    Map3.addLayer(brazil_pop1, {'min': 0, 'max': 10, 'palette': ['green', 'yellow', 'red']}, 'Brazil 2000')
    Map3.addLayer(brazil_pop2, {'min': 0, 'max': 10, 'palette': ['green', 'yellow', 'red']}, 'Brazil 2005')
    Map3.addLayer(brazil_pop3, {'min': 0, 'max': 10, 'palette': ['green', 'yellow', 'red']}, 'Brazil 2010'),
    Map3.addLayer(brazil_pop4, {'min': 0, 'max': 10, 'palette': ['green', 'yellow', 'red']}, 'Brazil 2015'),
    Map3.addLayer(brazil_pop5, {'min': 0, 'max': 10, 'palette': ['green', 'yellow', 'red']}, 'Brazil 2020'),
    Map3.addLayer(ba_clip, {'min': 0, 'max': 10, 'palette': ['black']}, 'Burned')

    # Show the map
    Map3.to_streamlit(height=700)

with tab4:
    # Initialize the header and the map
    st.header("Land Cover Difference")
    Map4 = geemap.Map(center=[-9, -51], zoom=4)

    # Add the Landcover image to the map
    diff = ee.Image('MODIS/006/MCD12Q1/2020_01_01').select('LC_Type1').subtract(ee.Image('MODIS/006/MCD12Q1/2003_01_01').select('LC_Type1'))
    lc_difference = diff.clip(brazil_shapefile)
    Map4.addLayer(lc_difference, paramVis, 'Differences between 2020 and 2003 ')

    # Create the map legend
    Map4.add_legend(
        title="Land cover",
        builtin_legend='MODIS/006/MCD12Q1',
        position='bottomright',
    )

    # Show the map
    Map4.to_streamlit(height=700)


Finally, we run our initialized Streamlit app, across Google Colab notebook with the code below

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com